<a href="https://colab.research.google.com/github/julianovale/PO240_Meta_heuristica/blob/main/PO240_Semana03_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
! pip install docplex
! pip install cplex

# Problema:

## Variáveis de decisão:


$x_1$: Quantidade do produto 1 (P1); </br>
$x_2$: Quantidade do produto 2 (P2); </br>

## Restrições:

*s.t.:*</br>
$x_1 \leq 3$ </br>
$x_2 \leq 4$ </br>
$x_1 + 2x_2 \leq 9$ </br>
$x_1, x_2 \geq 0$ </br>

## Função objetivo:

$Max\ Z = 5x_1 + 2x_2$ 

In [19]:
# Solução relaxada - restrições uma a uma

from docplex.mp.model import Model

plc = Model(name='Plano de Produção - PL - CPLEX')

x1 = plc.continuous_var(0, None, "x1")
x2 = plc.continuous_var(0, None, "x2")

plc.add_constraint(1*x1+0*x2 <= 3)
plc.add_constraint(0*x1+1*x2 <= 4)
plc.add_constraint(1*x1+2*x2 <= 9)

plc.maximize(5*x1+2*x2)

plc.solve()

print("FO = ", plc.objective_value)
print("x1 = ", x1.solution_value)
print("x2 = ", x2.solution_value)

FO =  21.0
x1 =  3.0
x2 =  3.0


In [20]:
# Matriz de restrições

from docplex.mp.model import Model

plic2 = Model(name='Plano de Produção - PLI - CPLEX')

nvar = 2
nrest = 3

# Matriz A vezes o vetor x
# A*x <= b

A = [[1,0],[0,1],[1,2]] # Matriz de coeficientes
b = [4, 4, 9] # Vetor de resultados
c = [5.00, 2.00] # Coeficientes da função objetivo

# Criando o vetor das variáveis
x = []
for j in range(nvar):
  tmp = plic2.continuous_var(0,None,'x'+str(j))
  x.append(tmp)

for i in range(nrest):
  plic2.add_constraint(plic2.sum(A[i][j]*x[j] for j in range(nvar)) <= b[i])

# Z = ct*x ('c' transposto vezes 'x') ==> Função objetivo
plic2.maximize(plic2.sum(c[j]*x[j] for j in range(nvar)))

# solução:
plic2.solve()

print("FO = ", plic2.objective_value)
for j in range(nvar):
  print("Quantidade de P%d"%(j+1)," = ", x[j].solution_value)

FO =  25.0
Quantidade de P1  =  4.0
Quantidade de P2  =  2.5


In [21]:
# Solução inteira - Matriz de restrições

from docplex.mp.model import Model

plic2 = Model(name='Plano de Produção - PLI - CPLEX')

nvar = 2
nrest = 3

# Matriz A vezes o vetor x
# A*x <= b

A = [[1,0],[0,1],[1,2]] # Matriz de coeficientes
b = [4, 4, 9] # Vetor de resultados
c = [5.00, 2.00] # Coeficientes da função objetivo

# Criando o vetor das variáveis
x = []
for j in range(nvar):
  tmp = plic2.integer_var(0,None,'x'+str(j)) # resolve mas é caro do ponto de vista computacional (o algoritmo não é o Simplex)
  x.append(tmp) 

# Restrições
for i in range(nrest):
  plic2.add_constraint(plic2.sum(A[i][j]*x[j] for j in range(nvar)) <= b[i])

# Z = ct*x ('c' transposto vezes 'x') ==> Função objetivo
plic2.maximize(plic2.sum(c[j]*x[j] for j in range(nvar)))

# solução:
plic2.solve()

print("FO = ", plic2.objective_value)
for j in range(nvar):
  print("Quantidade de P%d"%(j+1)," = ", x[j].solution_value)

FO =  24.0
Quantidade de P1  =  4.0
Quantidade de P2  =  2.0


In [1]:
! pip install ortools

In [14]:
# Usando o solver do ORTools para variáveis inteiras (Solver: CBC)

from ortools.linear_solver import pywraplp

pli3 = pywraplp.Solver('Plano de Produção - PLI - CBC', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

nvar = 2
nrest = 3

A = [[1,0],[0,1],[1,2]] # Matriz de coeficientes
b = [4, 4, 9] # Vetor de resultados
c = [5.00, 2.00] # Coeficientes da função objetivo

# Variáveis de decisão
x = []
for j in range(nvar):
  tmp = pli3.IntVar(0,pli3.infinity(),'x'+str(j))
  x.append(tmp)

# Restrições
for i in range(nrest):
  pli3.Add(pli3.Sum(A[i][j]*x[j] for j in range(nvar)) <= b[i])

# Função objetivo
pli3.Maximize(pli3.Sum(c[j]*x[j] for j in range(nvar)))

# Solução:
pli3.Solve()

# Relatório:
print("FO = ", pli3.Objective().Value())
for j in range(nvar):
  print("Quantidade de P%d"%(j+1)," = ", x[j].solution_value())

FO =  24.0
Quantidade de P1  =  4.0
Quantidade de P2  =  2.0
